<a href="https://colab.research.google.com/github/NjoodJ/RAG_With_Langchain/blob/main/RAG_With_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tiktoken
import openai
import os
import tqdm
import pinecone
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import CohereEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.memory import ConversationTokenBufferMemory
from langchain.prompts import (ChatPromptTemplate,SystemMessagePromptTemplate,)

/opt/anaconda3/envs/HMT/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
urls = [
"https://www.arabnews.com/category/tags/riyadh",
"https://www.akhbaar24.com/",
"https://www.timeanddate.com/weather/saudi-arabia/riyadh/ext"]

loader = WebBaseLoader(urls)
data = loader.load()
data

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRiyadh | Arab News\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nArab News\n\n\nArab News PK\nArab News JP\nArab News FR\n\n\n\n\nTuesday .\xa0 March 19, 2024  \n\n\n\n\n\n\nSearch form\n\nSearch \n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSaudi ArabiaNews\nThe Place\nThe Space\nWho's Who\nKSA Today\nGreen & Blue\n\nMiddle East\nWorld\nBusinessInsight\nCorporate News\n\nMediaMEDIA POWER LISTS\n\nLifestyleArt & Culture\nfashion\nOffbeat\nTravel\nEntertainment\nFood & Health\nThe Six\nBooks & Gaming\n\nOpinionEditorial\n\nSportSaudi Football\nFootball\nMotorsport\nTennis\nCricket\nGolf\nE-sport\nPodcast\n\nResearch & StudiesResearch Papers\nYouGov\nDeep Dive \nPreachers of Hate\n\nPodcast\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSaudi ArabiaNews\nThe Place\nThe Space\nWho's Who\nKSA Today\nGreen & Blue\n

In [ ]:
encoding_name = tiktoken.get_encoding("cl100k_base")
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=25)
docs = text_splitter.split_documents(data)

In [ ]:
embeddings = CohereEmbeddings(model='embed-multilingual-v3.0',# for english (embed-english-v3.0)
                              cohere_api_key='COHERE_API_KEY')

llm = ChatOpenAI(openai_api_key = "OPENAI_API_KEY"
                 ,max_tokens=800, temperature=0.7, verbose=True)

pinecone.init(
    api_key = 'PINECONE_ API_KEY',
    environment="gcp-starter",
)
index_name = "data"

docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

/opt/anaconda3/envs/HMT/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:

docsearch = Pinecone.from_existing_index(index_name, embeddings)

retriever = docsearch.as_retriever(search_kwargs={"k": 4})

compressor = CohereRerank(cohere_api_key = 'COHERE_API_KEY')
reranker = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
_template = """
Question:
{question}

Search query:
"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
Test = """
Question: {question}
---------------------
    {summaries}
---------------------
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(Test)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

In [ ]:
memory = ConversationTokenBufferMemory(llm=llm, memory_key="chat_history", return_messages=True, input_key='question', max_token_limit=1000)
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT, verbose=True)
answer_chain = load_qa_with_sources_chain(llm, chain_type="stuff", verbose=True,prompt=chat_prompt)

chain = ConversationalRetrievalChain(
            retriever=reranker,
            question_generator=question_generator,
            combine_docs_chain=answer_chain,
            verbose=False,
            memory=memory,
            rephrase_question=False
)

In [ ]:
query1 = "ماهو الطقس في الرياض"
query2 = "What is the summary of the discussion of cybersecurity issues at the Riyadh forum?"
query3 = "بماذا تعاهد ولي العهد مع بوتين عن منظمة الاوبك ؟"

In [ ]:
result = chain({"question": query1})
result

/opt/anaconda3/envs/HMT/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
Question: ماهو الطقس في الرياض
---------------------
    Content:  temperatures°CLast 2 weeks of weatherSee weather overview
2 Week Extended Forecast in Riyadh, Saudi ArabiaScroll right to see more ConditionsComfortPrecipitationSunDay TemperatureWeatherFeels LikeWind HumidityChanceAmountUVSunriseSunsetTue19 Mar25 / 20 °CShowers late. Overcast.26 °C13 km/h↑56%24%0.8  mm3 (Moderate)05:5718:04Wed20 Mar27 / 16 °CSunny.26 °C19 km/h↑23%23%0.2  mm7 (High)05:5618:04Thu21 Mar28 / 12 °CIncreasing cloudiness.26 °C16 km/h↑21%0%-3 (Moderate)05:5518:04Fri22 Mar29 / 16 °CPassing showers. Overcast.28 °C27 km/h↑27%52%5.0  mm3 (Moderate)05:5418:05Sat23 Mar29 / 17 °CAfternoon clouds.28 °C18 km/h↑33%14%0.3  mm5 (Moderate)05:5318:05Sun24 Mar32 / 20 °CShowers early. Breaks of sun late.30 °C19 km/h↑15%23%0.9  mm5 (Moderate)05:5218:06Mon25 Mar32 / 20 °CMorning clouds.29 °C20 km/h↑13%0%-7 (High)05

{'question': 'ماهو الطقس في الرياض',
 'chat_history': [HumanMessage(content='ماهو الطقس في الرياض'),
  AIMessage(content='حالياً في الرياض، درجة الحرارة تتراوح بين 25 و 20 درجة مئوية مع تساقط الأمطار والجو غائم.')],
 'answer': 'حالياً في الرياض، درجة الحرارة تتراوح بين 25 و 20 درجة مئوية مع تساقط الأمطار والجو غائم.'}

In [ ]:
result = chain({"question": query2})
result



> Entering new LLMChain chain...
Prompt after formatting:

Question:
What is the summary of the discussion of cybersecurity issues at the Riyadh forum?

Search query:




> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
Question: What is the summary of the discussion of cybersecurity issues at the Riyadh forum?
---------------------
    Content:  to over 3,700 emergency calls in Madinah following start of Umrah season Biden tells Netanyahu Rafah operation would be ‘mistake’ Soothing and soulful voices of the Grand Mosque in Makkah China discovers 100m tonne oilfield in Bohai Sea Ramadan nights: a fusion of spirituality and community Saudi crown prince receives written message from Uzbekistan’s president Emirates Red Crescent extends relief project in Gaza for Ramadan 
 
 







You are hereHomeRiyadh 







Riyadh












 
 



Experts to discuss cybersecurity issues at Riyadh forum


By
Arab News

·
18 March 2024









 
 



Artists explore ‘Saudi Futurism’ at Diriyah Contemporary Art Biennale in Riyadh


By
Jasmine Bager

·
15 March 2024 








 
 



Swe

{'question': 'What is the summary of the discussion of cybersecurity issues at the Riyadh forum?',
 'chat_history': [HumanMessage(content='ماهو الطقس في الرياض'),
  AIMessage(content='حالياً في الرياض، درجة الحرارة تتراوح بين 25 و 20 درجة مئوية مع تساقط الأمطار والجو غائم.'),
  HumanMessage(content='What is the summary of the discussion of cybersecurity issues at the Riyadh forum?'),
  AIMessage(content='The summary of the discussion of cybersecurity issues at the Riyadh forum is that experts gathered to discuss cybersecurity matters at the forum held in Riyadh on March 18, 2024.')],
 'answer': 'The summary of the discussion of cybersecurity issues at the Riyadh forum is that experts gathered to discuss cybersecurity matters at the forum held in Riyadh on March 18, 2024.'}

In [ ]:
result = chain({"question": query3})
result



> Entering new LLMChain chain...
Prompt after formatting:

Question:
بماذا تعاهد ولي العهد مع بوتين عن منظمة الاوبك ؟

Search query:




> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
Question: بماذا تعاهد ولي العهد مع بوتين عن منظمة الاوبك ؟
---------------------
    Content: 


أخبار 24 | آخر الأخبار وأهم المستجدات اليوم

















































 






أخبار 24 - الرئيسية





















تسجيل الدخول



 | 





 | 










                     
                    

 | 

                    19 مارس 2024
                






                                متابعات
                            



                                تجارة وأسواق
                            



                                دوليات
                            



                                منوعات
                            
Source: https://www.akhbaar24.com/

Content: 


أخبار 24 | آخر الأخبار وأهم المستجدات اليوم

















































 






أخبار 24 - الرئيسية





















تسجيل

{'question': 'بماذا تعاهد ولي العهد مع بوتين عن منظمة الاوبك ؟',
 'chat_history': [HumanMessage(content='ماهو الطقس في الرياض'),
  AIMessage(content='حالياً في الرياض، درجة الحرارة تتراوح بين 25 و 20 درجة مئوية مع تساقط الأمطار والجو غائم.'),
  HumanMessage(content='What is the summary of the discussion of cybersecurity issues at the Riyadh forum?'),
  AIMessage(content='The summary of the discussion of cybersecurity issues at the Riyadh forum is that experts gathered to discuss cybersecurity matters at the forum held in Riyadh on March 18, 2024.'),
  HumanMessage(content='بماذا تعاهد ولي العهد مع بوتين عن منظمة الاوبك ؟'),
  AIMessage(content='ولي العهد تعهد مع الرئيس الروسي فلاديمير بوتين بضرورة تعزيز التعاون بين السعودية وروسيا داخل منظمة البلدان المصدرة للبترول "أوبك"، وذلك خلال اتصال هاتفي جمع بينهما.')],
 'answer': 'ولي العهد تعهد مع الرئيس الروسي فلاديمير بوتين بضرورة تعزيز التعاون بين السعودية وروسيا داخل منظمة البلدان المصدرة للبترول "أوبك"، وذلك خلال اتصال هاتفي جمع بينهما.'}